# ECG Split 1D-CNN using PySyft (Client Side)

This code is the client part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
from pathlib import Path
import h5py
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

import syft as sy

print(torch.__version__)
print(sy.__version__)

1.8.1+cu102
0.5.0


In [2]:
project_path = Path.cwd().parent
project_path

PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/splitlearningcomputer/code/Users/dkn.work/split-learning-1D-HE')

In [3]:
data_dir = 'mitdb'
train_name = 'train_ecg.hdf5'
test_name = 'test_ecg.hdf5'
all_name = 'all_ecg.hdf5'

model_dir = 'model'
model_name = 'conv2'
model_ext = '.pth'

csv_dir = 'csv'
csv_ext = '.csv'

csv_name = 'conv2'
csv_accs_name = 'accs_conv2'

## Define ECG dataset class

In [4]:
class ECG(Dataset):
    def __init__(self, mode='train'):
        if mode == 'train':
            with h5py.File(project_path/data_dir/train_name, 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        elif mode == 'test':
            with h5py.File(project_path/data_dir/test_name, 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
        else:
            raise ValueError('Argument of mode should be train or test')
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

## Make train and test dataset batch generator

In [5]:
train_dataset = ECG(mode='train')
test_dataset = ECG(mode='test')

In [6]:
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))
print(f'type of train_loader: {type(train_loader)}')
print(f'type of test_loader: {type(test_loader)}')

type of train_loader: <class 'torch.utils.data.dataloader.DataLoader'>
type of test_loader: <class 'torch.utils.data.dataloader.DataLoader'>


Use `batch_size=len(train_dataset)` to get the whole dataset

In [7]:
x_train, y_train = next(iter(train_loader))
x_test, y_test = next(iter(test_loader))
print(f'x_train: {type(x_train)}, {x_train.size()}')
print(f'y_train: {type(y_train)}, {y_train.size()}')
print(f'x_test: {type(x_test)}, {x_test.size()}')
print(f'y_test: {type(y_test)}, {y_test.size()}')

x_train: <class 'torch.Tensor'>, torch.Size([13245, 1, 128])
y_train: <class 'torch.Tensor'>, torch.Size([13245])
x_test: <class 'torch.Tensor'>, torch.Size([13245, 1, 128])
y_test: <class 'torch.Tensor'>, torch.Size([13245])


In [8]:
x_train.device

device(type='cpu')

## Upload the pointers of the dataset to duet so that the server can search, but cannot view or retrieve the data without the client's permission

In [9]:
# duet = sy.duet(loopback=True)
duet = sy.duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: 332b215d80ccbd33aeedfc2ce640ba00

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("332b215d80ccbd33aeedfc2ce640ba00")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



In [10]:
x_train.send(duet, pointable=True, tags=["x_train"],
                                    description="training input data")
y_train.send(duet, pointable=True, tags=["y_train"],
                                    description="training output data")
x_test.send(duet, pointable=True, tags=["x_test"],
                                    description="testing input data")
y_test.send(duet, pointable=True, tags=["y_test"],
                                    description="testing output data")

[2021-09-03T08:45:21.343534+0000][CRITICAL][logger]][13006] You do not have permission to .get() Object with ID: <UID: 8d5ca6451cfd44f28d08d509da157840>Please submit a request.
[2021-09-03T08:45:21.344111+0000][CRITICAL][logger]][13006] You do not have permission to .get() Object with ID: <UID: 8d5ca6451cfd44f28d08d509da157840>Please submit a request.


Add `action="accept"` handler for convenience. 
The server should only ask to access the activation signals and the ground truth labels

In [11]:
duet.requests.add_handler(action="accept")

[2021-09-03T08:47:12.623166+0000][CRITICAL][logger]][13006] Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same
[2021-09-03T08:47:12.623777+0000][CRITICAL][logger]][13006] Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same
Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...be the same')>) at /anaconda/envs/SyftEnv2/lib/python3.9/site-packages/pyee/_asyncio.py:57
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...be the same')>) at /anaconda/envs/SyftEnv2/lib/python3.9/site-packages/pyee/_asyncio.py:57>
Traceback (most recent call last):
  File "/anaconda/envs/SyftEnv2/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/anaconda/envs/SyftEnv2/lib/python3.9/site-packages/pyee/_asyncio.py", line 64, in _callback
    self.emit("error", exc)
  File "/anaconda/envs/SyftEnv2/lib/python3.9/sit